In [3]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from utils import SRC_DIR
from concurrent.futures import ThreadPoolExecutor

# def is_cropped(row):
#     file_path = Path(row.file_path)
#     cropped_file_path = SRC_DIR / file_path.parent / 'cropped' / file_path.name
#     if cropped_file_path.exists():
#         return True
#     else:
#         return False
# 
# 
# def replace_file_path(row):
#     file_path = Path(row.file_path)
#     return (file_path.parent / 'cropped' / file_path.name).as_posix()
# 
# for dataset_path in [
#     'ai_gen_dataset_5_cats',
#     'ai_gen_dataset_10_cats',
#     'medium_one_hot',
#     'small_one_hot'
# ]:
#     df = pd.read_csv(f'datasets/{dataset_path}.csv', index_col=0)
#     start = df.shape[0]
#     df['is_cropped'] = df.apply(is_cropped, axis=1)
#     df = df[df['is_cropped']]
#     df['file_path'] = df.apply(replace_file_path, axis=1)
#     df.drop(['is_cropped'], axis=1, inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     df.to_csv(f"datasets/cropped_{dataset_path}.csv")
#     print(dataset_path, "reduced by", abs(df.shape[0] - start))

In [4]:
import shutil

temp = Path("./runpod_images")
temp.mkdir(exist_ok=True, parents=True)

In [5]:
df_name = 'medium_one_hot'
df = pd.read_csv(f'datasets/cropped_{df_name}.csv', index_col=0)

In [7]:
def copy_file(row):
    T = temp / row.file_path.replace("/cropped/", "/")
    T.parent.mkdir(parents=True, exist_ok=True)
    if not T.exists():
        shutil.copy(
            src=SRC_DIR / row.file_path,
            dst=T
        )


with ThreadPoolExecutor(max_workers=16) as executor:
    futures = []
    for i, row in df.iterrows():
        futures.append(
            executor.submit(copy_file, row)
        )

    for f in tqdm(futures, total=len(futures), desc="copying files"):
        f.result()

copying files: 100%|██████████| 206486/206486 [00:05<00:00, 40551.93it/s]


In [8]:
df.shape

(206486, 114)